In [5]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt


def haversine(lon1, lat1, lon2, lat2):
   
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
   
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
  
    r = 6371
    return c * r

training_data = pd.read_csv("../../data/processed/extended_features/training_extended_features.csv", sep="\t")
airports = pd.read_csv("../../data/Other/airports.csv", sep=",")
airports.info()
training_data[['DepartureAirport', 'ArrivalAirport']] = training_data['Route'].str.split('-', expand=True)

airport_coords = {

}


# Funkcja pomocnicza
def get_distance(row):
    dep = row['DepartureAirport']
    arr = row['ArrivalAirport']
    if dep in airport_coords and arr in airport_coords:
        lat1, lon1 = airport_coords[dep]
        lat2, lon2 = airport_coords[arr]
        return haversine(lon1, lat1, lon2, lat2)
    else:
        return np.nan

# Obliczenie odległości
training_data['RouteDistance'] = training_data.apply(get_distance, axis=1)

# Sprawdzenie brakujących wartości
missing_airports = training_data[training_data['RouteDistance'].isnull()][['DepartureAirport', 'ArrivalAirport']].drop_duplicates()

print("Lotniska do uzupełnienia:")
print(missing_airports)

unique_airports = pd.unique(
    pd.concat([missing_airports['DepartureAirport'], missing_airports['ArrivalAirport']])
)

print(f"Liczba unikalnych lotnisk: {len(unique_airports)}")
print(unique_airports)




for airport in unique_airports:
    row = airports[airports['iata_code'] == airport]
    if not row.empty:
        lat = row['latitude_deg'].values[0]
        lon = row['longitude_deg'].values[0]
        airport_coords[airport] = (lat, lon)
    elif airport == "SXF":
        airport_coords[airport] = (52.384444, 13.522222)
        print("SFX Berlin-Schönefeld ")


training_data['RouteDistance'] = training_data.apply(get_distance, axis=1)
training_data.drop(columns=['Route'])
training_data.to_csv("../../data/processed/extended_features/training_with_distances.csv", index=False, sep="\t")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83017 entries, 0 to 83016
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 83017 non-null  int64  
 1   ident              83017 non-null  object 
 2   type               83017 non-null  object 
 3   name               83017 non-null  object 
 4   latitude_deg       83017 non-null  float64
 5   longitude_deg      83017 non-null  float64
 6   elevation_ft       68534 non-null  float64
 7   continent          43895 non-null  object 
 8   iso_country        82749 non-null  object 
 9   iso_region         83017 non-null  object 
 10  municipality       78343 non-null  object 
 11  scheduled_service  83017 non-null  object 
 12  icao_code          7850 non-null   object 
 13  iata_code          9097 non-null   object 
 14  gps_code           43238 non-null  object 
 15  local_code         35701 non-null  object 
 16  home_link          428